In [1]:
import spacy



In [2]:
!python -m spacy download en_core_web_lg

2023-09-22 16:55:15.521041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [18]:
nlp = spacy.load("en_core_web_lg")

In [19]:
import spacy
from spacy.tokens import DocBin

doc_bin = DocBin().from_disk("/content/spacy_train_data_resolved.spacy")
count = 0  # Counter to keep track of the number of printed documents

for doc in doc_bin.get_docs(spacy.blank("en").vocab):
    print(f"Document Text: {doc}")
    print("Entities:")
    for ent in doc.ents:
        print(f"  Start: {ent.start_char}, End: {ent.end_char}, Label: {ent.label_}")
    count += 1
    print("-----------")
    if count >= 50:  # Stop after printing 5 documents
        break


Document Text: enhances transparency market trends conditions conclusions made appraiser.
Entities:
  Start: 22, End: 35, Label: Mortgage Terms
  Start: 36, End: 46, Label: Mortgage Terms
  Start: 64, End: 73, Label: Mortgage Terms
-----------
Document Text: hybrid appraisal involves appraiser.
Entities:
  Start: 26, End: 35, Label: Mortgage Terms
-----------
Document Text: appraisal form needed subject property 2 4 unit dwelling borrower using rental income qualify. fhlmc 72 fnma fhlmc 72
Entities:
  Start: 0, End: 14, Label: Mortgage Terms
  Start: 57, End: 65, Label: Mortgage Terms
  Start: 72, End: 85, Label: Mortgage Terms
  Start: 86, End: 93, Label: Mortgage Terms
-----------
Document Text: u.s. income tax return estates trusts. report income, deductions, gains, losses, etc. estate trust.
Entities:
  Start: 5, End: 11, Label: Mortgage Terms
  Start: 39, End: 45, Label: Mortgage Terms
  Start: 46, End: 52, Label: Mortgage Terms
  Start: 54, End: 64, Label: Mortgage Terms
  Start:

In [26]:
!python -m spacy debug data config.cfg

2023-09-22 17:30:49.249894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
1221 training docs
1221 evaluation docs
⚠ 1167 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (1221)

============================== Vocab & Vectors ==============================
ℹ 62649 total word(s) in the data (5026 unique)
ℹ 514157 vectors (514157 unique keys, 300 dimensions)
⚠ 1177 words in training data without vectors (2%)

========================== Named Entity Recognition ==========================
ℹ 43 label(s)
0 missing value(s) (tokens with '-' label)
⚠ Low number of examples for label 'Lock Extension Policy - Broker'
(23)
⚠ Low number of example

In [27]:
!python -m spacy train config.cfg --output ./ --paths.train ./spacy_train_data_resolved.spacy --paths.dev ./spacy_train_data_resolved.spacy

2023-09-22 17:32:21.903231: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     98.45    0.00    0.00    0.00    0.00
  0     200       1936.18   6032.65    0.37   43.75    0.18    0.00
  0     400        469.48   6355.11    0.08  100.00    0.04    0.00
  1     600        832.90   6599.50    1.08    6.36    0.59    0.01
  1     800       1422.28   7902.90    0.52   30.77    0.26    0.01
  2    1000       1742.27   9320.78    2.36   36.90    1.22    0.02
  3    1200       4062.42  10544.34   20.08   31

In [28]:
nlp_ner = spacy.load("model-last")

In [34]:
import spacy
import pandas as pd

# Load your trained spaCy model
model_directory = '/content/model-last'
nlp = spacy.load(model_directory)

# Load your test CSV (replace with the actual path)
test_csv_path = '/content/model_best_test.csv'
test_df = pd.read_csv(test_csv_path)


# Initialize an empty list to store the predicted text entities for each row
predicted_text_entities_list = []

# Iterate through the DataFrame and populate the predicted text entities list
for index, row in test_df.iterrows():
    doc = nlp(row['summarized_text'])
    predicted_text_entities = [doc.text[ent.start_char:ent.end_char] for ent in doc.ents]
    predicted_text_entities_list.append(predicted_text_entities)

# Add the 'predicted_text_entities' column to the DataFrame
test_df['predicted_text_entities'] = predicted_text_entities_list

# Save the DataFrame with predicted text entities (optional)
output_csv_path = '/content/model_best_text_entities.csv'
test_df.to_csv(output_csv_path, index=False)